In [80]:
# !pip install --upgrade mlflow

In [1]:
import pickle
import mlflow

In [2]:
mlflow.__version__

'2.13.1'

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("exp_1")
mlflow.set_experiment_tag("base regression model version","1.0")

In [4]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

In [5]:
train_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
val_path = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

In [6]:
df_train = pd.read_parquet(train_path) # for training 
df_val = pd.read_parquet(val_path) # for validation and test

In [7]:
# df = pd.concat([df_jan,df_feb],axis=0)
df = df_train

In [8]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [9]:
len(df.columns)

19

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [11]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60

In [12]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3.066766e+06,3066766,3066766,2.995023e+06,3.066766e+06,2.995023e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,2.995023e+06,2.995023e+06,3.066766e+06
mean,1.730215e+00,2023-01-17 00:22:26.288164,2023-01-17 00:38:06.427874,1.362532e+00,3.847342e+00,1.497440e+00,1.663980e+02,1.643926e+02,1.194483e+00,1.836707e+01,1.537842e+00,4.882900e-01,3.367941e+00,5.184907e-01,9.820847e-01,2.702038e+01,2.274231e+00,1.074086e-01,1.566900e+01
min,1.000000e+00,2008-12-31 23:01:42,2009-01-01 14:29:11,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-9.000000e+02,-7.500000e+00,-5.000000e-01,-9.622000e+01,-6.500000e+01,-1.000000e+00,-7.510000e+02,-2.500000e+00,-1.250000e+00,-2.920000e+01
25%,1.000000e+00,2023-01-09 16:21:57.250000,2023-01-09 16:37:06,1.000000e+00,1.060000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.540000e+01,2.500000e+00,0.000000e+00,7.116667e+00
50%,2.000000e+00,2023-01-17 08:42:29.500000,2023-01-17 08:58:30.500000,1.000000e+00,1.800000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.720000e+00,0.000000e+00,1.000000e+00,2.016000e+01,2.500000e+00,0.000000e+00,1.151667e+01
75%,2.000000e+00,2023-01-24 16:26:27,2023-01-24 16:42:49,1.000000e+00,3.330000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.200000e+00,0.000000e+00,1.000000e+00,2.870000e+01,2.500000e+00,0.000000e+00,1.830000e+01
max,2.000000e+00,2023-02-01 00:56:53,2023-02-02 09:28:47,9.000000e+00,2.589281e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,1.160100e+03,1.250000e+01,5.316000e+01,3.808000e+02,1.969900e+02,1.000000e+00,1.169400e+03,2.500000e+00,1.250000e+00,1.002918e+04
std,4.438480e-01,NaN,NaN,8.961200e-01,2.495838e+02,6.474767e+00,6.424413e+01,6.994368e+01,5.294762e-01,1.780782e+01,1.789592e+00,1.034641e-01,3.826759e+00,2.017579e+00,1.833529e-01,2.216359e+01,7.718454e-01,3.556511e-01,4.259435e+01


In [13]:
categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype('str')

In [14]:
len(df['duration'][(df['duration']>=1) & (df['duration']<=60)])/len(df['duration'])

0.9812202822125979

In [15]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [16]:
train_data = df[categorical].to_dict(orient='records')

In [17]:
dv = DictVectorizer()

In [18]:
X_train = dv.fit_transform(train_data)

In [19]:
print(X_train.shape)

(3009173, 515)


# Creating and training a model

In [20]:
target = 'duration'
y_train = df[target].values

In [21]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [22]:
y_train_predict = model.predict(X_train)

In [23]:
mse(y_train,y_train_predict,squared=False)

7.649261927686161

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

## Function to modify the dataframe to get the validation set

In [25]:
class PrepareVal():

    def __init__(self,min_t:int,max_t:int,categorical:list,target:str):
        '''
        class to modify the dataframe based on the constraints on the feature columns and the 
        trip duration times 

        Parameters: 
            min_t : minimumt time for acceptable duration in minutes
            max-t : maximum time for acceptable duration in minutes
            categorical : columns accepted as categorical columns that will act as the feature columns
            target : column name for the target variables 
        
        Returns:

        transformed df 
        
        y_val - validation target values
        '''

        self.min_t = min_t
        self.max_t = max_t
        self.categorical = categorical
        
        return None 

    def transform(self,df):
        
        df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60
        
        df = df[(df['duration'] >= self.min_t)&(df['duration'] <= self.max_t)].copy()

        df[self.categorical] = df[self.categorical].astype('str')

        val_df = df[self.categorical].to_dict(orient='records')

        X_val = dv.fit_transform(val_df)
        y_val = df[target].values

        return X_val,y_val

        

In [26]:
val_func = PrepareVal(1,60,categorical,'target')

In [27]:
X_val,y_val = val_func.transform(df_val)

In [28]:
model.fit(X_val,y_val)

LinearRegression()

In [29]:
y_val_pred = model.predict(X_val)

In [30]:
print(f'Validation Error {mse(y_val,y_val_pred,squared=False)}')

Validation Error 7.778948571132272


In [31]:
# !pip install hyperopt xgboost lightgbm

In [32]:
with mlflow.start_run():

    mlflow.set_tag("author","chakresh")
    mlflow.log_param("train-data-path",train_path)
    mlflow.log_param("val-data-path",val_path)

    model = LinearRegression()

    model.fit(X_train,y_train)
    y_train_pred = model.predict(X_train)

    train_mse = mse(y_train,y_train_pred,squared=False)

    mlflow.log_metric("train mse",train_mse)
    


    

## Experiment tracking and hyperparameter tuning 

In [33]:
# !pip install xgboost==2.0.3

In [34]:
import xgboost as xgb

from hyperopt import fmin, tpe, STATUS_OK, hp, Trials
from hyperopt.pyll import scope

In [35]:
train = xgb.DMatrix(X_train,label=y_train)
val = xgb.DMatrix(X_val,label=y_val)

In [36]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        xgb_model = xgb.train(
                        params=params,
                        dtrain = train,
                        num_boost_round=500,
                        evals=[(val,"validation")],
                        early_stopping_rounds=50 
                    )
        y_val_pred = xgb_model.predict(val)
        loss = mse(y_val,y_val_pred,squared=False)
        mlflow.log_metric("mse",loss)
        return {"loss":loss, "status":STATUS_OK}

        

In [37]:
search_scope = {
            'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
            'learning_rate': hp.loguniform('learning_rate', -3, 0),
            'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
            'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
            'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
            'objective': 'reg:linear',
            'seed': 42,
            
        }

In [38]:
best_result = fmin(
    fn=objective,
    space=search_scope,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials(),
    verbose=False
)

/Users/chakreshsingh/opt/anaconda3/envs/mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:03:13] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.06084
[1]	validation-rmse:10.06025
[2]	validation-rmse:10.07328
[3]	validation-rmse:10.09047
[4]	validation-rmse:10.11186
[5]	validation-rmse:10.13285
[6]	validation-rmse:10.16663
[7]	validation-rmse:10.16232
[8]	validation-rmse:10.16551
[9]	validation-rmse:10.19319
[10]	validation-rmse:10.20015
[11]	validation-rmse:10.24255
[12]	validation-rmse:10.25250
[13]	validation-rmse:10.28544
[14]	validation-rmse:10.29807
[15]	validation-rmse:10.36900
[16]	validation-rmse:10.37943
[17]	validation-rmse:10.40787
[18]	validation-rmse:10.44029
[19]	validation-rmse:10.49849
[20]	validation-rmse:10.51724
[21]	validation-rmse:10.55123
[22]	validation-rmse:10.58401
[23]	validation-rmse:10.60505
[24]	validation-rmse:10.65104
[25]	validation-rmse:10.67368
[26]	validation-rmse:10.72123
[27]	validation-rmse:10.78412
[28]	validation-rmse:10.80875
[29]	validation-rmse:10.85588
[30]	validation-rmse:10.86201
[31]	validation-rmse:10.94432
[32]	validation-rmse:10.97183
[33]	validation-rmse

/Users/chakreshsingh/opt/anaconda3/envs/mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:03:31] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.30059
[1]	validation-rmse:10.31380
[2]	validation-rmse:11.00240
[3]	validation-rmse:11.37768
[4]	validation-rmse:11.82881
[5]	validation-rmse:11.98105
[6]	validation-rmse:12.10770
[7]	validation-rmse:12.63531
[8]	validation-rmse:12.73176
[9]	validation-rmse:12.88804
[10]	validation-rmse:13.27205
[11]	validation-rmse:13.84128
[12]	validation-rmse:14.03113
[13]	validation-rmse:14.22280
[14]	validation-rmse:15.02993
[15]	validation-rmse:15.06906
[16]	validation-rmse:15.23623
[17]	validation-rmse:15.32859
[18]	validation-rmse:15.51290
[19]	validation-rmse:15.75144
[20]	validation-rmse:15.83982
[21]	validation-rmse:16.03400
[22]	validation-rmse:16.38358
[23]	validation-rmse:16.56743
[24]	validation-rmse:17.00611
[25]	validation-rmse:17.10783
[26]	validation-rmse:17.24267
[27]	validation-rmse:17.25366
[28]	validation-rmse:17.29471
[29]	validation-rmse:17.35709
[30]	validation-rmse:17.49387
[31]	validation-rmse:17.68112
[32]	validation-rmse:17.74458
[33]	validation-rmse

/Users/chakreshsingh/opt/anaconda3/envs/mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:03:46] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


...

[0]	validation-rmse:10.27329
[1]	validation-rmse:10.28427
[2]	validation-rmse:11.01803
[3]	validation-rmse:11.38766
[4]	validation-rmse:11.63951
[5]	validation-rmse:11.91219
[6]	validation-rmse:12.70206
[7]	validation-rmse:12.76927
[8]	validation-rmse:13.05353
[9]	validation-rmse:13.44130
[10]	validation-rmse:13.77425
[11]	validation-rmse:14.11018
[12]	validation-rmse:14.48901
[13]	validation-rmse:14.52910
[14]	validation-rmse:14.62504
[15]	validation-rmse:15.37004
[16]	validation-rmse:15.45092
[17]	validation-rmse:15.73042
[18]	validation-rmse:15.79890
[19]	validation-rmse:15.81044
[20]	validation-rmse:15.93831
[21]	validation-rmse:15.99233
[22]	validation-rmse:16.20422
[23]	validation-rmse:16.23591
[24]	validation-rmse:16.32329
[25]	validation-rmse:16.85339
[26]	validation-rmse:17.11758
[27]	validation-rmse:17.22006
[28]	validation-rmse:17.38970
[29]	validation-rmse:17.53752
[30]	validation-rmse:17.55462
[31]	validation-rmse:17.90540
[32]	validation-rmse:18.04517
[33]	validation-rmse

/Users/chakreshsingh/opt/anaconda3/envs/mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:05:37] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.61124
[1]	validation-rmse:11.15432
[2]	validation-rmse:12.26608
[3]	validation-rmse:13.13481
[4]	validation-rmse:13.56198
[5]	validation-rmse:13.69163
[6]	validation-rmse:14.59024
[7]	validation-rmse:15.03239
[8]	validation-rmse:16.36798
[9]	validation-rmse:16.63862
[10]	validation-rmse:16.71077
[11]	validation-rmse:16.81911
[12]	validation-rmse:16.86454
[13]	validation-rmse:16.90302
[14]	validation-rmse:17.33058
[15]	validation-rmse:17.55102
[16]	validation-rmse:18.06122
[17]	validation-rmse:18.25214
[18]	validation-rmse:18.30049
[19]	validation-rmse:18.37445
[20]	validation-rmse:18.38848
[21]	validation-rmse:18.58921
[22]	validation-rmse:18.68100
[23]	validation-rmse:18.57009
[24]	validation-rmse:18.62086
[25]	validation-rmse:18.71816
[26]	validation-rmse:18.76621
[27]	validation-rmse:18.79428
[28]	validation-rmse:18.73769
[29]	validation-rmse:18.88241
[30]	validation-rmse:18.94171
[31]	validation-rmse:18.97248
[32]	validation-rmse:18.78276
[33]	validation-rmse

/Users/chakreshsingh/opt/anaconda3/envs/mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:05:55] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.07486
[1]	validation-rmse:10.20459
[2]	validation-rmse:10.22391
[3]	validation-rmse:10.31948
[4]	validation-rmse:10.48489
[5]	validation-rmse:10.60744
[6]	validation-rmse:10.71070
[7]	validation-rmse:11.01860
[8]	validation-rmse:11.35683
[9]	validation-rmse:11.47764
[10]	validation-rmse:11.55162
[11]	validation-rmse:11.80720
[12]	validation-rmse:11.91644
[13]	validation-rmse:12.09171
[14]	validation-rmse:12.29183
[15]	validation-rmse:12.41369
[16]	validation-rmse:12.63012
[17]	validation-rmse:12.71253
[18]	validation-rmse:12.87573
[19]	validation-rmse:13.13532
[20]	validation-rmse:13.20074
[21]	validation-rmse:13.46607
[22]	validation-rmse:13.58629
[23]	validation-rmse:13.69411
[24]	validation-rmse:13.75241
[25]	validation-rmse:13.88159
[26]	validation-rmse:13.98024
[27]	validation-rmse:14.06715
[28]	validation-rmse:14.22990
[29]	validation-rmse:14.27841
[30]	validation-rmse:14.39714
[31]	validation-rmse:14.44889
[32]	validation-rmse:14.50199
[33]	validation-rmse

# Logging the best model run using autolog

In [42]:
best_params = {
    'learning_rate':	0.059603805067328826,
'max_depth':	24,
'min_child_weight':	1.766271760072879,
'objective'	:'reg:linear',
'reg_alpha'	:0.012657205291454728,
'reg_lambda' :	0.12170874907939165,
'seed':	42
}

In [43]:
mlflow.xgboost.autolog()
train = xgb.DMatrix(X_train,label=y_train)
val = xgb.DMatrix(X_val,label=y_val)
xgb_model = xgb.train(
                        params=best_params,
                        dtrain = train,
                        num_boost_round=500,
                        evals=[(val,"validation")],
                        early_stopping_rounds=50 
                    )

2024/06/03 01:13:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '22bb5b10bd8845de9ec151e1af0804b8', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/Users/chakreshsingh/opt/anaconda3/envs/mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:13:38] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.06490
[1]	validation-rmse:10.06705
[2]	validation-rmse:10.06917
[3]	validation-rmse:10.07500
[4]	validation-rmse:10.08213
[5]	validation-rmse:10.09354
[6]	validation-rmse:10.10344
[7]	validation-rmse:10.11992
[8]	validation-rmse:10.11965
[9]	validation-rmse:10.13519
[10]	validation-rmse:10.14861
[11]	validation-rmse:10.16692
[12]	validation-rmse:10.18483
[13]	validation-rmse:10.18665
[14]	validation-rmse:10.19403
[15]	validation-rmse:10.20640
[16]	validation-rmse:10.21823
[17]	validation-rmse:10.21464
[18]	validation-rmse:10.22703
[19]	validation-rmse:10.24413
[20]	validation-rmse:10.25229
[21]	validation-rmse:10.26407
[22]	validation-rmse:10.27672
[23]	validation-rmse:10.27407
[24]	validation-rmse:10.28369
[25]	validation-rmse:10.30578
[26]	validation-rmse:10.31330
[27]	validation-rmse:10.31038
[28]	validation-rmse:10.31877
[29]	validation-rmse:10.31395
[30]	validation-rmse:10.33245
[31]	validation-rmse:10.34701
[32]	validation-rmse:10.33012
[33]	validation-rmse

2024/06/03 01:13:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/chakreshsingh/opt/anaconda3/envs/mlflow/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [01:13:47] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
